<h1>Person re-identification Project</h1>
<p>
<h4>This notebook contains the code done as part of the major project at Pulchowk Campus.</h4>
Notebook contains code to train and test the model using Market1501 dataset.</p>

<i>Link to the kaggle dataset  
https://www.kaggle.com/rayiooo/reid_market-1501

**Contributor**: *Krishna Prasad Panthi*</i>

In [ ]:
from google.colab import files
files.upload()

{}

**Downloading data from kaggle**

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change permission
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pengcw1/market-1501

 95% 138M/146M [00:01<00:00, 85.6MB/s]
100% 146M/146M [00:01<00:00, 84.2MB/s]


**Extracting data to the dataset folder**

In [ ]:
!unzip market-1501.zip
!mv Market-1501-v15.09.15 dataset


**Import libraries**


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import pickle
import os



<h1>Load training data</h1>
Saves the training data in <i>"train_dataset.pkl"</i> file

In [ ]:
def load_trains(dataset_path):
    '''
    arguments:
    dataset_path : system path to the folder containing the datasets
                   assumes bounding_box_train contains the training images
                   and bounding_box_test contains the testing images

                   name of image file is assumed to be with imagelabels_whatever.jpg

    saves to train_dataset.pkl:
    X_train: 3D numpy array of all training images with images

    Y_train : 1D numpy array of labels corresponding to each label list.
              i.e. labels are not repeated.
    '''

    #get the system paths for the training image folder
    train_images_path = dataset_path + '/bounding_box_train'
    # train_images_path = dataset_path + '/gt_bbox'

    #sort the image files in order of labels
    train_images_list = sorted(os.listdir(train_images_path))

    X_train = []

    train_labels = []


    #loop through the image list
    for i in train_images_list:
        #check if file name ends with .jpg
        if i.split('.')[-1] == 'jpg':
            #read the image if true
            img = cv2.imread(os.path.join(train_images_path,i))
            X_train.append(img)
            #read the label from the file name
            train_labels.append(int(i.split('_')[0]))


    #convert all python lists to numpy arrays
    X_train = np.array(X_train)
    train_labels = np.array(train_labels)
    
    #split the images with same labels into different lists
    unique,indices,inverses,counts = np.unique(train_labels,return_index=True,return_inverse=True,return_counts=True)
    X_train = np.split(X_train,indices)
    del X_train[0]
    X_train = np.array(X_train)
    Y_train = np.unique(inverses)



    return X_train,Y_train

X_train,Y_train= load_trains('dataset')
file = open('train_dataset.pkl','wb')
pickle.dump([X_train,Y_train],file)
file.close()
print(X_train.shape)

(751,)


<h1>Load testing data</h1>
Saves the training data in <i>"test_dataset.pkl"</i> file

In [ ]:
def load_tests(dataset_path):
    '''
    arguments:
    dataset_path : system path to the folder containing the datasets
                   assumes bounding_box_test contains the testing images
                   and bounding_box_test contains the testing images

                   name of image file is assumed to be with imagelabels_whatever.jpg

    Returns:
    saves data to the pickle file test_dataset.pkl 
    test_images, test_images_names, query_images, query_images_name
    '''

    #get the system paths for the testing image folder
    test_images_path = dataset_path + '/bounding_box_test'

    #get the system path fot the testing image folder
    query_images_path = dataset_path + '/query'

    #sort the image files in order of labels
    test_images_list = os.listdir(test_images_path)
    query_images_list = os.listdir(query_images_path)
    del test_images_list[test_images_list.index('Thumbs.db')]
    del query_images_list[query_images_list.index('Thumbs.db')]


    test_img  = []
    query_img = []


    #loop through the image list
    for i in test_images_list:
        #check if file name ends with .jpg
        if i.split('.')[-1] == 'jpg':
            #read the image if true
            img = cv2.imread(os.path.join(test_images_path,i))
            test_img.append(img)
            


    for i in query_images_list:
        #check if file name end with .jpg and ignore files with labels 0 or -1
        if i.split('.')[-1] == 'jpg' and int(i.split('_')[0])!=-1 and int(i.split('_')[0])!=0:
            img = cv2.imread(os.path.join(query_images_path,i))
            query_img.append(img)

    #convert all python lists to numpy arrays
    test_img = np.array(test_img)
    query_img = np.array(query_img)
    test_img_names = np.array(test_images_list)
    query_img_names = np.array(query_images_list)


    return test_img,test_img_names,query_img,query_img_names


t,t_n,q,q_n = load_tests('dataset')
file = open('test_dataset.pkl','wb')
pickle.dump([t,t_n,q,q_n],file)
file.close()
print('done')

done


<h1>Loss Functions</h1>
<h4>Here we use combinations of three loss functions</h4>
<ul>
<li>Categorical cross entropy loss</li>
<li>Triplet loss</li>
<li>Weighted Contrastive loss <a href=https://arxiv.org/pdf/1811.01459.pdf>https://arxiv.org/pdf/1811.01459.pdf</a></li>
</ul>

In [ ]:
class Losses:
  centers = tf.Variable(tf.constant_initializer(0.0)(shape=[751,256],dtype=tf.float32),trainable=False,name='centers')
  
  @staticmethod
  def _pairwise_distances(embeddings, squared=False):
      """Compute the 2D matrix of distances between all the embeddings.

      Args:
          embeddings: tensor of shape (batch_size, embed_dim)
          squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                  If false, output is the pairwise euclidean distance matrix.

      Returns:
          pairwise_distances: tensor of shape (batch_size, batch_size)
      """
      # Get the dot product between all embeddings
      # shape (batch_size, batch_size)
      dot_product = tf.matmul(embeddings, tf.transpose(embeddings))

      # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
      # This also provides more numerical stability (the diagonal of the result will be exactly 0).
      # shape (batch_size,)
      square_norm = tf.linalg.diag_part(dot_product)

      # Compute the pairwise distance matrix as we have:
      # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
      # shape (batch_size, batch_size)
      distances = tf.expand_dims(square_norm, 0) - 2.0 * dot_product + tf.expand_dims(square_norm, 1)

      # Because of computation errors, some distances might be negative so we put everything >= 0.0
      distances = tf.maximum(distances, 0.0)

      if not squared:
          # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
          # we need to add a small epsilon where distances == 0.0
          mask = tf.cast(tf.equal(distances, 0.0),tf.float32)
          distances = distances + mask * 1e-16
          distances = tf.sqrt(distances)

          # Correct the epsilon added: set the distances on the mask to be exactly 0.0
          distances = distances * (1.0 - mask)

      return distances

  @staticmethod
  def _masked_maximum(data, mask, dim=1):
      axis_minimums = tf.math.reduce_min(data, dim, keepdims=True)
      masked_maximums = (
          tf.math.reduce_max(
              tf.math.multiply(data - axis_minimums, mask), dim, keepdims=True
          )
          + axis_minimums
      )
      return masked_maximums

  @staticmethod
  def _masked_minimum(data, mask, dim=1):
      axis_maximums = tf.math.reduce_max(data, dim, keepdims=True)
      masked_minimums = (
          tf.math.reduce_min(
              tf.math.multiply(data - axis_maximums, mask), dim, keepdims=True
          )
          + axis_maximums
      )
      return masked_minimums

  @staticmethod
  def triplet_semihard_loss(
      labels,
      embeddings,
      margin = 1.0,
      distance_metric = "L2",
  ) -> tf.Tensor:
      labels = tf.argmax(labels, axis=1)
      
      precise_embeddings = tf.cast(embeddings, tf.float32)

      # Reshape label tensor to [batch_size, 1].
      lshape = tf.shape(labels)
      labels = tf.reshape(labels, [lshape[0], 1])

      # Build pairwise squared distance matrix

      if distance_metric == "L1":
          pdist_matrix = Losses._pairwise_distances(
              precise_embeddings, squared=False
          )

      else:
          pdist_matrix = Losses._pairwise_distances(
              precise_embeddings, squared=True
          )

      # Build pairwise binary adjacency matrix.
      adjacency = tf.math.equal(labels, tf.transpose(labels))
      # Invert so we can select negatives only.
      adjacency_not = tf.math.logical_not(adjacency)

      batch_size = tf.size(labels)

      # Compute the mask.
      pdist_matrix_tile = tf.tile(pdist_matrix, [batch_size, 1])
      mask = tf.math.logical_and(
          tf.tile(adjacency_not, [batch_size, 1]),
          tf.math.greater(
              pdist_matrix_tile, tf.reshape(tf.transpose(pdist_matrix), [-1, 1])
          ),
      )
      mask_final = tf.reshape(
          tf.math.greater(
              tf.math.reduce_sum(
                  tf.cast(mask, dtype=tf.float32), 1, keepdims=True
              ),
              0.0,
          ),
          [batch_size, batch_size],
      )
      mask_final = tf.transpose(mask_final)

      adjacency_not = tf.cast(adjacency_not, dtype=tf.float32)
      mask = tf.cast(mask, dtype=tf.float32)

      # negatives_outside: smallest D_an where D_an > D_ap.
      negatives_outside = tf.reshape(
          Losses._masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size]
      )
      negatives_outside = tf.transpose(negatives_outside)

      # negatives_inside: largest D_an.
      negatives_inside = tf.tile(
          Losses._masked_maximum(pdist_matrix, adjacency_not), [1, batch_size]
      )
      semi_hard_negatives = tf.where(mask_final, negatives_outside, negatives_inside)

      loss_mat = tf.math.add(margin, pdist_matrix - semi_hard_negatives)

      mask_positives = tf.cast(adjacency, dtype=tf.float32) - tf.linalg.diag(
          tf.ones([batch_size])
      )

      # In lifted-struct, the authors multiply 0.5 for upper triangular
      #   in semihard, they take all positive pairs except the diagonal.
      num_positives = tf.math.reduce_sum(mask_positives)

      triplet_loss = tf.math.truediv(
          tf.math.reduce_sum(
              tf.math.maximum(tf.math.multiply(loss_mat, mask_positives), 0.0)
          ),
          num_positives,
      )
      return tf.cast(triplet_loss, embeddings.dtype)
      



  @staticmethod
  def osm_caa_loss(embeddings,labels,weights,alpha=1.2,sigma=0.8,l_parameter=0.5):

      '''
      alpha : margin of weighted contrastive loss, as mentioned in the paper 
      l     : hyperparameter controlling weights of positive set and the negative set  
      sigma :  sigma OSM (0.8) as mentioned in paper

      embeddings : feature vector : (n x d)
      labels : (n,)
      weights : Fully Connected weights of classification layer (dxC), C is the number of classes: represents the vectors for class
      returns online soft mining class attention aware adjusted contrastive loss 
      '''

      labels = tf.argmax(labels, axis=1)
      n = labels.shape[0]
      n = 32
      dist = Losses._pairwise_distances(embeddings)

      p_mask = tf.cast(tf.equal(labels[:, tf.newaxis], labels[tf.newaxis, :]), tf.float32)

      n_mask = 1- p_mask

      S = tf.exp(-dist / (sigma**2 ) )
      S_N = tf.clip_by_value(tf.nn.relu(alpha - dist), clip_value_min=tf.constant(1e-12) , clip_value_max=tf.constant(1e12)) 
      S_P = S * p_mask
      S_N = S_N * n_mask
      S  = S_P + S_N
      weights = tf.math.l2_normalize(weights, axis=0)
      denom = tf.reduce_sum(tf.exp(tf.matmul(embeddings , weights)),1)
      num =  tf.exp (tf.reduce_sum( embeddings * tf.transpose(tf.gather(weights , labels , axis=1)) , 1 ))

      atten_class = num / denom

      
      temp = tf.tile(tf.expand_dims(atten_class, 0),[n,1])

      A =  tf.math.minimum(temp , tf.transpose(temp))

      W = S * A
      W_P = W * p_mask
      W_N = W * n_mask
      W_P = W_P * (1 - tf.eye(n))
      W_N = W_N * (1 - tf.eye(n))

      L_P =  tf.reduce_sum(W_P * tf.pow(dist,2) ) / (2 * tf.reduce_sum(W_P) )
      L_N  = tf.reduce_sum(W_N * tf.pow(S_N , 2) ) / (2 * tf.reduce_sum(W_N) )
      loss = (1-l_parameter) * L_P + l_parameter * L_N

      return loss

  @staticmethod
  def get_center_loss(features, labels, alpha, num_classes):
    labels = tf.argmax(labels, axis=1)
    labels = tf.reshape(labels, [-1])
    labels = tf.cast(labels,tf.int32)

    centers_batch = tf.gather(Losses.centers, labels)

    #loss = tf.nn.l2_loss(features - centers_batch)

    diff = centers_batch - features
    diff = (1 - alpha)*(centers_batch-features)
    labels = tf.reshape(labels, [32, 1])
    Losses.centers = tf.tensor_scatter_nd_sub(Losses.centers,labels,diff)
    loss = tf.reduce_mean(tf.square(features-centers_batch))
    # unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)

    # appear_times = tf.gather(unique_count, unique_idx)
    # appear_times = tf.reshape(appear_times, [-1, 1])

    # diff = diff / tf.cast((1 + appear_times), tf.float32)
    # loss = tf.reduce_sum(tf.abs(diff))
    # diff = alpha * diff

    #labels=tf.reshape(labels, [32, 1])
    #Losses.centers = tf.tensor_scatter_nd_sub(Losses.centers, labels, diff)
    return loss

<h1>Utility functions</h1>

In [ ]:
def one_hot_encoding(labels):
  size = np.max(labels)+1
  encode = np.eye(size)[labels]
  return encode

def get_batches(tr_img,tr_lb):
  '''randomly select 4 images each of 8 classes'''
  temp = np.arange(0,751)
  #randomly select 8 out of 1501 indices
  eight_indices = np.random.choice(temp,8,replace=False)
  #randomly select 8 labels
  labels = tr_lb[eight_indices]
  #repeat them 4 times to get 32 labels
  labels = np.repeat(labels,4,axis=0)
  #initialize train list
  train = []
  #for each class of image selected according to
  #8 random indices, select 4 random images
  for i in tr_img[eight_indices]:
      x = np.arange(i.shape[0])
      try:
          train.append(i[np.random.choice(x,4,replace=False)])
      except:
          train.append(i[np.random.choice(x,4,replace=True)])

  #reshape the selected 32 images into 4D tensor
  train = np.array(train).reshape(32,128,64,3)

  #generate 32 indices
  x=np.arange(0,32)
  #shuffle the 32 indices
  np.random.shuffle(x)
  #shuffle the 32 training images
  train = train[x]
  #shuffle the 32 training labels
  labels = labels[x]
  return train.astype('float32'),labels.astype('float32')

<h1>Model and Parameters</h1>
<b>EfficientNetB0</b> model trained on ImageNet dataset is used as the baseline model. <a href=https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html>Efficient Net link</a><br>
A 256 layer dense layer is attached to the output of convolution layers from <b>EfficientNetB0</b> model.

<ul>
<li>Input Images shape : 128 x 64 x 3</li>
<li>Batch size : 32</li>
<li>Beta Ratio : 0.3 <br><i>This is used to weigh losses source: <a href=https://arxiv.org/pdf/1912.05295v1.pdf>https://arxiv.org/pdf/1912.05295v1.pdf</a></i></li>
<ul>


In [ ]:
BATCH_SIZE = 32
BETA_RATIO = 0.3
IMG_SIZE = (128, 64)
IMG_SHAPE = IMG_SIZE + (3,)
file = open('train_dataset.pkl','rb')
X_train,Y_train = pickle.load(file)

train_images = X_train/255.0
train_labels = one_hot_encoding(Y_train)
num_classes = train_images.shape[0]



base_model = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')



inputs = tf.keras.Input(shape=(128, 64, 3))
x = base_model(inputs, training=True)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256)(x)
out1 =tf.math.l2_normalize(x,axis=1)
x = tf.keras.layers.ReLU()(x)
out2 = tf.keras.layers.Dense(num_classes,activation='softmax')(x)
model = tf.keras.Model(inputs,[out1,out2])

#model.layers[-1].weights[0].shape

<h1>Combined Loss function</h1>

In [ ]:
def custom_loss(model,x,y,training):
  output =  model(x, training=training)
  #cross_entropy_loss
  classification_loss = tf.reduce_mean(tf.keras.losses.CategoricalCrossentropy()(y,output[1]))
  
  #triplet_loss
  triplet_loss = Losses.triplet_semihard_loss(labels=y,embeddings=output[0],margin=1.0)
  
  #osm_caa_loss
  osm_caa_loss = Losses.osm_caa_loss(output[0],y,model.layers[-1].weights[0])
 
  #center_loss
  center_loss = Losses.get_center_loss(output[0],y,0.5,751)
  
  total_loss = (1-BETA_RATIO)*triplet_loss + classification_loss + BETA_RATIO*osm_caa_loss + 0.3*center_loss
  return total_loss

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = custom_loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)



<h1>Optimizer</h1>
Adam optimzer with the learning rate of 10^(-3) and default parameters is used.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)


<h1>Training</h1>
The model was trained for 25 epochs


In [ ]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 25

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  
  # Training loop - using batches of 32

  for i in range(int(16000/BATCH_SIZE)):
    x_batch,y_batch = get_batches(train_images,train_labels)
    # Optimize the model
    loss_value, grads = grad(model, x_batch, y_batch)
    if i %100 == 0:
      print('epoch : ',epoch, ' i : ',i,' loss : ',loss_value)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(np.argmax(y_batch,axis=1), model(x_batch, training=True)[1])

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 1 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

epoch :  0  i :  0  loss :  tf.Tensor(7.9116387, shape=(), dtype=float32)
epoch :  0  i :  100  loss :  tf.Tensor(7.7166877, shape=(), dtype=float32)
epoch :  0  i :  200  loss :  tf.Tensor(7.3374968, shape=(), dtype=float32)
epoch :  0  i :  300  loss :  tf.Tensor(6.8200884, shape=(), dtype=float32)
epoch :  0  i :  400  loss :  tf.Tensor(6.5597672, shape=(), dtype=float32)
Epoch 000: Loss: 7.156, Accuracy: 6.275%
epoch :  1  i :  0  loss :  tf.Tensor(6.8179426, shape=(), dtype=float32)
epoch :  1  i :  100  loss :  tf.Tensor(4.524348, shape=(), dtype=float32)
epoch :  1  i :  200  loss :  tf.Tensor(5.4620004, shape=(), dtype=float32)
epoch :  1  i :  300  loss :  tf.Tensor(4.8847513, shape=(), dtype=float32)
epoch :  1  i :  400  loss :  tf.Tensor(5.916371, shape=(), dtype=float32)
Epoch 001: Loss: 5.657, Accuracy: 22.094%
epoch :  2  i :  0  loss :  tf.Tensor(3.9640744, shape=(), dtype=float32)
epoch :  2  i :  100  loss :  tf.Tensor(3.4657428, shape=(), dtype=float32)
epoch :  2  i

In [ ]:
!mkdir -p saved_model
model.save('saved_model/cl_model.h5')
!zip -r /content/file.zip /content/saved_model/
from google.colab import files
files.download("/content/file.zip")

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/cl_model.h5 (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
x_batch,y_batch = get_batches(train_images,train_labels)
np.argmax(y_batch,axis=1)==np.argmax(model(x_batch,training=False)[1],axis=1)



array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [ ]:
#Testing code(outputs rank 1 accuracy)
BATCH_SIZE = 200

def find_l2_norm(a,b):
    '''
    a : 2D numpy array
    b : 2D numpy array
    returns the L2_norm between each vector of a and each vector of b
    if a : 4 x 256 and b : 7 x 256
       output is 4 x 7 norm matrix

     '''
    try:
        a = a.reshape([1,256])
    except:
        pass
    try:
        b = b.reshape([1,256])
    except:
        pass
    dot_product = np.matmul(a, np.transpose(b))
    a = np.square(a)
    b = np.square(b)
    norm = np.sum((np.expand_dims(a,axis=1) + b), axis=2) - 2*dot_product
    norm = np.sqrt(norm)
    return norm

def stats(correct,incorrect):
    correct = np.around(correct,2)
    incorrect = np.around(incorrect,2)
    mean_correct,mean_incorrect = np.mean(correct),np.mean(incorrect)
    md_correct,md_incorrect = np.median(correct),np.median(incorrect)
    sd_correct,sd_incorrect = np.std(correct),np.std(incorrect)
    min_correct,max_correct = np.min(correct),np.max(correct)
    min_incorrect,max_incorrect = np.min(incorrect),np.max(incorrect)

    print('\n\n')
    print('Correct distance')
    print('mean_correct : ',mean_correct)
    print('sd_correct : ',sd_correct)
    print('md_correct : ',md_correct)
    print('min : ',min_correct, '  ','max : ',max_correct)
    print('val : ', (np.sum(np.cast['int'](correct<0.33)) / np.sum(np.cast['int'](correct>0.0)))*100 )

    print('\n\n')
    print('Incorrect distance')
    print('mean_incorrect : ',mean_incorrect)
    print('sd_incorrect : ',sd_incorrect)
    print('md_incorrect : ',md_incorrect)
    print('min : ',min_incorrect, '  ','max : ',max_incorrect)
    print('\n\n')



def market_test():

    '''
    output : ouputs the rank-1 accuracy
    
    '''
    file=open('test_dataset.pkl','rb')
    test_mat,test_imgs,query_mat,query_imgs = pickle.load(file)
    test_embed=get_embeddings(test_mat)
    query_embed= get_embeddings(query_mat)
    test_embed = np.array(test_embed)
    query_embed = np.array(query_embed)
    print('no problems with embeddings')
    print(test_mat.shape)
    print(query_mat.shape)
    #counts the number of correct query images identified
    correct  = 0
    #counts the total number of query images
    tot = 0 
    #keeps track of query embeddings to be tested
    counter = 0
    #number of embeddings which are tested at once
    batch_size = 100

    correct_dist=[]
    incorrect_dist=[]

    tot_num = query_embed.shape[0]

    while(counter+batch_size < tot_num):
        distances = find_l2_norm(query_embed[counter:counter+batch_size],test_embed)
        imgs = query_imgs[counter:counter+batch_size]
        for i in range(distances.shape[0]):
            dis = distances[i].reshape([-1])
            count = zip(dis,test_imgs)
            count2 = sorted(count,key=lambda x:x[0])
            try:
              if int(imgs[i].split('_')[0]) == int(count2[0][1].split('_')[0]):
                  correct += 1
                  correct_dist.append(count2[0][0])
            except:
              print('hello')
            else:
                incorrect_dist.append(count2[0][0])
            tot += 1
        counter += batch_size
        print(correct)
    
    distances = find_l2_norm(query_embed[counter:tot_num],test_embed)
    imgs = query_imgs[counter:tot_num]
    for i in range(distances.shape[0]):
        dis = distances[i].reshape([-1])
        count = zip(dis,test_imgs)
        count2 = sorted(count,key=lambda x:x[0])
        if int(imgs[i].split('_')[0]) == int(count2[0][1].split('_')[0]):
            correct += 1
            correct_dist.append(count2[0][0])
        else:
            incorrect_dist.append(count2[0][0])
        tot += 1
    counter += batch_size

    stats(correct_dist,incorrect_dist)

    print('rank 1 accuracy : ',(correct/tot)*100,'%')


def get_embeddings(imgs):
  '''
  imgs : numpy array of list of images of shape [n X 128 x 64 x 3]
  outputs : outputs the embeddings after passing through the 
            base model
  '''
  embeddings = []
  count = 0
  tot_size = imgs.shape[0]
  print(tot_size)

  while (count + BATCH_SIZE ) < tot_size :
    x = imgs[count:count+BATCH_SIZE]
    output = model.predict(x)[0]
    embeddings+=list(output)
    count += BATCH_SIZE

  x = imgs[count:tot_size]
  
  output = model.predict(x)[0]
  embeddings+=list(output)
  print(len(embeddings))
  return embeddings


market_test()

19732
19732
3368
3368
no problems with embeddings
(19732, 128, 64, 3)
(3368, 128, 64, 3)
96
188
279
369
466
561
655
751
845
937
1034
1128
1226
1319
1413
1507
1602
1698
1792
1887
1984
2080
2169
2268
2364
2458
2553
2648
2743
2840
2936
3033
3124



Correct distance
mean_correct :  0.47260353
sd_correct :  0.121952936
md_correct :  0.46
min :  0.2    max :  0.9
val :  9.284818067754077



Incorrect distance
mean_incorrect :  0.48527846
sd_incorrect :  0.13193332
md_incorrect :  0.47
min :  0.2    max :  0.9



rank 1 accuracy :  94.65558194774347 %


In [ ]:
!mkdir -p saved_model
!mkdir -p saved_model
model.save('saved_model/my_model.h5')

In [ ]:
!zip -r /content/file.zip /content/saved_model/
from google.colab import files
files.download("/content/file.zip")

In [ ]:
model.save('my_model.h5')

In [ ]:
!ls

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
model = tf.keras.models.load_model('my_model.h5')
files = os.listdir('.')
imgs = []
labels = []
for img in files:
  try:
    if img.split('.')[1]=='jpg':
      mat = cv2.imread(img)
      imgs.append(mat)
      labels.append(int(img.split('_')[0]))
  except:
    print('ok')
imgs=np.array(imgs)
imgs.shape

In [ ]:
features = model.predict(imgs)
features[0].shape

In [ ]:
def find_l2_norm(a,b):
    '''
    a : 2D numpy array
    b : 2D numpy array
    returns the L2_norm between each vector of a and each vector of b
    if a : 4 x 256 and b : 7 x 256
       output is 4 x 7 norm matrix

     '''
    try:
        a = a.reshape([1,256])
    except:
        pass
    try:
        b = b.reshape([1,256])
    except:
        pass
    dot_product = np.matmul(a, np.transpose(b))
    a = np.square(a)
    b = np.square(b)
    norm = np.sum((np.expand_dims(a,axis=1) + b), axis=2) - 2*dot_product + 1e-6
    norm = np.sqrt(norm)
    return norm

out = find_l2_norm(features[0],features[0])
out

In [ ]:
labels
